## Task 2

The closing root tag was initially missing from the .xml,
It has been added manually.

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
# naive 1st try with Pandas - we can't see most of the data
df = pd.read_xml('nested_xml_example.xml')
df

,id,alertTime,intensity,title,text,sources,attachments,comments,state,alertCount,startTime,participants,messages,alertType,attributes
0,xmas-20200914-1,2020-09-14T23:59:59.999-0400,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,NaN,NaN,NaN,NaN,1,2020-09-14T23:59:59.999-0400,NaN,NaN,5652434329,NaN
1,xmas-20200915-1,2020-09-15T23:59:59.999-0400,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,NaN,NaN,NaN,NaN,1,2020-09-15T23:59:59.999-0400,NaN,NaN,5652434329,NaN
2,yyyy-20201208-1,2020-12-08T23:59:59.999-0500,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,NaN,NaN,NaN,NaN,1,2020-12-08T23:59:59.999-0500,NaN,NaN,5652434329,NaN
3,yyyy-20201209-1,2020-12-09T23:59:59.999-0500,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,NaN,NaN,NaN,NaN,1,2020-12-09T23:59:59.999-0500,NaN,NaN,5652434329,NaN
4,yyyy-20201210-1,2020-12-10T23:59:59.999-0500,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,NaN,NaN,NaN,NaN,1,2020-12-10T23:59:59.999-0500,NaN,NaN,5652434329,NaN
5,yyyy-20201211-1,2020-12-11T23:59:59.999-0500,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,NaN,NaN,NaN,NaN,1,2020-12-11T23:59:59.999-0500,NaN,NaN,5652434329,NaN


In [3]:
tree = ET.parse('nested_xml_example.xml')

In [4]:
root = tree.getroot()

In [5]:
# Aggregate the data to a dataframe out of which we will build our usable tabular data/
#these aggregation lists have to be defined outside the function since it is recursive 
tags=[]
levels =[]
texts = []
attr = []
isparent = []
def dig (elem, level = 0):
    if len(list(elem))>0:
        print(level*'-' + elem.tag )
        tags.append(elem.tag)
        levels.append(level)
        texts.append(elem.text)
        attr.append(elem.attrib)
        isparent.append(True)
        for child in list(elem):
            dig (child, level+1)
    else:
        print(level*'-' + elem.tag )
        tags.append(elem.tag)
        levels.append(level)
        texts.append(elem.text)
        attr.append(elem.attrib)
        isparent.append(False)    

In [6]:
dig(root)

alerts
-alert
--alertTime
--intensity
--title
--text
--sources
---source
----market
----country
----target
--attachments
--comments
---comment
----time
----author
----text
----alertStatus
----alertAssignee
--state
---alertStatus
---alertAssignee
--alertCount
--startTime
--participants
---house
--messages
--alertType
--attributes
---item
---item
---item
---item
---item
-alert
--alertTime
--intensity
--title
--text
--sources
---source
----market
----country
----target
--attachments
--comments
---comment
----time
----author
----text
----alertStatus
----alertAssignee
--state
---alertStatus
---alertAssignee
--alertCount
--startTime
--participants
---house
--messages
--alertType
--attributes
---item
---item
---item
---item
---item
-alert
--alertTime
--intensity
--title
--text
--sources
---source
----market
----country
----target
--attachments
--comments
---comment
----time
----author
----text
----alertStatus
----alertAssignee
--state
---alertStatus
---alertAssignee
--alertCount
--startTime
-

We observe that there are multiple elements with the tag 'item' in each nested structure - these will need to be stored in a seprate df as we cannot nest in pandas dfs 

In [7]:
# we make a df to bring together important information about the all the elements in the .xml
# - out of this we will construct our final data structure.
df_source = pd.DataFrame({'level':levels, 'tag':tags, 'text':texts, 'att': attr , 'isparent': isparent})
df_source

,level,tag,text,att,isparent
0,0,alerts,\n,{},True
1,1,alert,\n,{'id': 'xmas-20200914-1'},True
2,2,alertTime,2020-09-14T23:59:59.999-0400,{},False
3,2,intensity,0,{},False
4,2,title,AFTER PROCESSING COMPLETE (HOUSE),{},False
...,...,...,...,...,...
194,3,item,None,"{'key': 'ALERT-PROCESSING-START-TIME', 'value'...",False
195,3,item,None,"{'key': 'ALERT-PROCESSING-START-DATE', 'value'...",False
196,3,item,None,"{'key': 'ALERT-PROCESSING-END-TIME', 'value': ...",False
197,3,item,None,"{'key': 'ALERT-PROCESSING-END-DATE', 'value': ...",False


In [8]:
max_depth = max(levels) # of the nested .xml

In [9]:
# Get the names of the tags which exist in each level of nesting
def get_tags(df, lvl):
    tags = df[df['level']==lvl].tag.unique()
    return tags.tolist()

# Make a dataframe with all the data which exist in each level of nesting.
# Ultimately, combining these dfs will yield an aggregation of the data in .xml
# We can only consider tags which appear once for each level of nesting, thus we exclude 'item' at this stage.
def make_dfs_from_levels (df,tags, lvl):
    # remove item tag
    if 'item' in tags:
        tags.pop(tags.index('item')) 
    
    df2 = df[(df['tag']==tags[0])& (df['level']==lvl)][['text','att']]
    df2.reset_index(drop=True, inplace=True)
    df2.rename({'text': tags[0]+'_text', 'att': tags[0]+'_att'}, axis=1, inplace=True)
    if len(tags)>1:
        for t in tags[1:]:
            df_next = df[(df['tag']==t)& (df['level']==lvl)][['text','att']]
            df_next.reset_index(drop=True, inplace=True)
            df_next.rename({'text': t+'_text', 'att': t+'_att'}, axis=1, inplace=True)
            df2 = pd.concat([df2,df_next],axis =1)
    return df2

In [10]:
# example -> the .xml data from the 1st level of nesting
level = 1
tags = get_tags(df_source, level)
df_example = make_dfs_from_levels(df_source,tags, level)
df_example

,alert_text,alert_att
0,\n,{'id': 'xmas-20200914-1'}
1,\n,{'id': 'xmas-20200915-1'}
2,\n,{'id': 'yyyy-20201208-1'}
3,\n,{'id': 'yyyy-20201209-1'}
4,\n,{'id': 'yyyy-20201210-1'}
5,\n,{'id': 'yyyy-20201211-1'}


In [11]:
# Now, repeat the above step for all levels of nesting
level = 1
tags = get_tags(df_source, level)
df = make_dfs_from_levels(df_source,tags, level)
for level in range(2, max_depth+1):
    tags = get_tags(df_source, level)
    df_next_lvl = make_dfs_from_levels(df_source,tags, level)
    #df_next_lvl.reset_index(drop=True, inplace=True)
    df = pd.concat([df,df_next_lvl],axis =1)
    
df   

,alert_text,alert_att,alertTime_text,alertTime_att,intensity_text,intensity_att,title_text,title_att,text_text,text_att,...,time_text,time_att,author_text,author_att,text_text,text_att,alertStatus_text,alertStatus_att,alertAssignee_text,alertAssignee_att
0,\n,{'id': 'xmas-20200914-1'},2020-09-14T23:59:59.999-0400,{},0,{},AFTER PROCESSING COMPLETE (HOUSE),{},AFTER PROCESSING COMPLETE (HOUSE) - This alert...,{},...,2020-09-15T14:56:37.977+0000,{},joeblogs,{},"Assigned to ""joeblogs"". Changed status to ""Inf...",{},Information Alert Only,{},joeblogs,{}
1,\n,{'id': 'xmas-20200915-1'},2020-09-15T23:59:59.999-0400,{},0,{},AFTER PROCESSING COMPLETE (HOUSE),{},AFTER PROCESSING COMPLETE (HOUSE) - This alert...,{},...,2020-09-21T17:49:18.123+0000,{},eltonjoh,{},"Assigned to ""eltonjoh"". Changed status to ""Inf...",{},Information Alert Only,{},eltonjoh,{}
2,\n,{'id': 'yyyy-20201208-1'},2020-12-08T23:59:59.999-0500,{},0,{},AFTER PROCESSING COMPLETE (HOUSE),{},AFTER PROCESSING COMPLETE (HOUSE) - This alert...,{},...,2020-12-29T16:13:03.611+0000,{},apeters,{},"Changed status to ""Information Alert Only"".",{},Information Alert Only,{},None,{}
3,\n,{'id': 'yyyy-20201209-1'},2020-12-09T23:59:59.999-0500,{},0,{},AFTER PROCESSING COMPLETE (HOUSE),{},AFTER PROCESSING COMPLETE (HOUSE) - This alert...,{},...,2020-12-29T16:13:03.611+0000,{},apeters,{},"Changed status to ""Information Alert Only"".",{},Information Alert Only,{},None,{}
4,\n,{'id': 'yyyy-20201210-1'},2020-12-10T23:59:59.999-0500,{},0,{},AFTER PROCESSING COMPLETE (HOUSE),{},AFTER PROCESSING COMPLETE (HOUSE) - This alert...,{},...,2021-01-04T17:47:45.746+0000,{},joeblogs,{},"Assigned to ""joeblogs"". Changed status to ""Inf...",{},Information Alert Only,{},joeblogs,{}
5,\n,{'id': 'yyyy-20201211-1'},2020-12-11T23:59:59.999-0500,{},0,{},AFTER PROCESSING COMPLETE (HOUSE),{},AFTER PROCESSING COMPLETE (HOUSE) - This alert...,{},...,2021-01-04T17:47:45.746+0000,{},joeblogs,{},"Assigned to ""joeblogs"". Changed status to ""Inf...",{},Information Alert Only,{},joeblogs,{}


In [12]:
# df contains all the data from the .xml
# We see that many of the columns are empty or contain empty dictionaries
# we can filter them out and make a leaner df if we wish
df_lean = df.copy()

df_lean = df_lean.dropna(axis=1, how='all')
df_lean = df_lean.loc[:, ~df_lean.where(df_lean.astype(bool)).isna().all(axis=0)]
df_lean

,alert_text,alert_att,alertTime_text,intensity_text,title_text,text_text,sources_text,comments_text,state_text,alertCount_text,...,alertAssignee_text,house_text,house_att,market_text,country_text,time_text,author_text,text_text,alertStatus_text,alertAssignee_text
0,\n,{'id': 'xmas-20200914-1'},2020-09-14T23:59:59.999-0400,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,\n,\n,\n,1,...,joeblogs,015,{'marketCode': 'xmas'},xmas,Bost,2020-09-15T14:56:37.977+0000,joeblogs,"Assigned to ""joeblogs"". Changed status to ""Inf...",Information Alert Only,joeblogs
1,\n,{'id': 'xmas-20200915-1'},2020-09-15T23:59:59.999-0400,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,\n,\n,\n,1,...,eltonjoh,015,{'marketCode': 'xmas'},xmas,Bost,2020-09-21T17:49:18.123+0000,eltonjoh,"Assigned to ""eltonjoh"". Changed status to ""Inf...",Information Alert Only,eltonjoh
2,\n,{'id': 'yyyy-20201208-1'},2020-12-08T23:59:59.999-0500,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,\n,\n,\n,1,...,None,15,{'marketCode': 'yyyy'},yyyy,Bost,2020-12-29T16:13:03.611+0000,apeters,"Changed status to ""Information Alert Only"".",Information Alert Only,None
3,\n,{'id': 'yyyy-20201209-1'},2020-12-09T23:59:59.999-0500,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,\n,\n,\n,1,...,None,15,{'marketCode': 'yyyy'},yyyy,Bost,2020-12-29T16:13:03.611+0000,apeters,"Changed status to ""Information Alert Only"".",Information Alert Only,None
4,\n,{'id': 'yyyy-20201210-1'},2020-12-10T23:59:59.999-0500,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,\n,\n,\n,1,...,joeblogs,15,{'marketCode': 'yyyy'},yyyy,Bost,2021-01-04T17:47:45.746+0000,joeblogs,"Assigned to ""joeblogs"". Changed status to ""Inf...",Information Alert Only,joeblogs
5,\n,{'id': 'yyyy-20201211-1'},2020-12-11T23:59:59.999-0500,0,AFTER PROCESSING COMPLETE (HOUSE),AFTER PROCESSING COMPLETE (HOUSE) - This alert...,\n,\n,\n,1,...,joeblogs,15,{'marketCode': 'yyyy'},yyyy,Bost,2021-01-04T17:47:45.746+0000,joeblogs,"Assigned to ""joeblogs"". Changed status to ""Inf...",Information Alert Only,joeblogs


In [13]:
# finally we store the 'item' elements in a seperate df, as there are 5 of them for each row of the main df.
df_item_tags = df_source[df_source['tag']=='item']
# we use the 'alert_att' tag as a top-level link between the data in the main df and
# the df which stores the 'item' elements.
# First, we expand the df['alert_att'] values *5 to match the number of rows in df_item_tags
# which stores the 'item' elements and then we add is as a column in the df_item_tags.
link =[[i]*5 for i in df['alert_att'].tolist()]
link = [item for sublist in link for item in sublist]
df_item_tags['alert_att'] = link
df_item_tags

/var/folders/4t/qx8dpw5j639g89v8gtkmd7tr0000gn/T/ipykernel_49487/3113142709.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_item_tags['alert_att'] = link


,level,tag,text,att,isparent,alert_att
29,3,item,None,"{'key': 'ALERT-PROCESSING-START-TIME', 'value'...",False,{'id': 'xmas-20200914-1'}
30,3,item,None,"{'key': 'ALERT-PROCESSING-START-DATE', 'value'...",False,{'id': 'xmas-20200914-1'}
31,3,item,None,"{'key': 'ALERT-PROCESSING-END-TIME', 'value': ...",False,{'id': 'xmas-20200914-1'}
32,3,item,None,"{'key': 'ALERT-PROCESSING-END-DATE', 'value': ...",False,{'id': 'xmas-20200914-1'}
33,3,item,None,"{'key': 'NUMBER-OF-ALERTS', 'value': '1'}",False,{'id': 'xmas-20200914-1'}
62,3,item,None,"{'key': 'ALERT-PROCESSING-START-TIME', 'value'...",False,{'id': 'xmas-20200915-1'}
63,3,item,None,"{'key': 'ALERT-PROCESSING-START-DATE', 'value'...",False,{'id': 'xmas-20200915-1'}
64,3,item,None,"{'key': 'ALERT-PROCESSING-END-TIME', 'value': ...",False,{'id': 'xmas-20200915-1'}
65,3,item,None,"{'key': 'ALERT-PROCESSING-END-DATE', 'value': ...",False,{'id': 'xmas-20200915-1'}
66,3,item,None,"{'key': 'NUMBER-OF-ALERTS', 'value': '1'}",False,{'id': 'xmas-20200915-1'}
